In [1]:
# Import functions from the module
from netcdf_tools import get_points, get_values

In [2]:
import pandas as pd

df = pd.read_csv('../data/trajectories.csv')

trayectories_df = pd.DataFrame(df)
print(trayectories_df.shape)

# Delecciona disinct dates
dates = trayectories_df['date'].unique()

dates_df = pd.DataFrame(dates, columns=['date'])

import matplotlib.pyplot as plt

#plt.figure(figsize=(12, 8))
#plt.scatter(trayectories_df['longitude'], trayectories_df['latitude'], s=0.1)


(176767, 9)


In [5]:

# Selecciona netCDF de acuerdo a las fechas del dataframe
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

def select_netCDf(date):
    path = '../data/sst'
    # Cambia formato de yyyy-mm-dd a yyyymmdd
    date = date.replace('-', '')
    filename = f"{path}/{date}090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_clipped.nc"
    print(filename)
    # If filename exists the open it and get lat, lon and SST, else SST = 'NA'
    nc_file = nc.Dataset(filename)
    print(nc_file.variables.keys())
    # Get lat and lon
    lat = nc_file.variables['lat'][:]
    lon = nc_file.variables['lon'][:]
    # Get SST
    sst = nc_file.variables['analysed_sst'][0][:,:]
    sst = sst - 273.15
    # Close netCDF
    nc_file.close()
    return lon, lat, sst


In [6]:
def plot_netCDF(date):
    lon, lat, sst = select_netCDf(date)
    # Plot netCDF
    #plt.pcolor(lon, lat, sst)
    #plt.title(f"Sea Surface Temperature {date}")

fecha = dates_df['date'][3]
plot_netCDF(fecha)

../data/sst/20140122090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_clipped.nc
dict_keys(['time', 'lat', 'lon', 'analysed_sst', 'analysis_error', 'mask', 'sea_ice_fraction'])


In [8]:
# Get points
puntos = get_points(trayectories_df, fecha)
puntos.head()

,id,geom,date,latitude,longitude,name,season,spheroid_dist_to_colony,sst
1251,281,POINT(-118.291402 28.883883),2014-01-22,28.883883,-118.291405,LAAL01,incubacion,0.0,17.746
1252,282,POINT(-118.291442 28.883833),2014-01-22,28.883833,-118.291440,LAAL01,incubacion,0.0,17.746
1253,283,POINT(-118.291393 28.883812),2014-01-22,28.883812,-118.291390,LAAL01,incubacion,0.0,17.746
1254,284,POINT(-118.291473 28.883857),2014-01-22,28.883858,-118.291470,LAAL01,incubacion,0.0,17.746
1255,285,POINT(-118.29144 28.88382),2014-01-22,28.883820,-118.291440,LAAL01,incubacion,0.0,17.746


In [9]:
import numpy as np
# Obten temperaturas de netCDF a partir de coordenadas de puntos

lon, lat, sst = select_netCDf(fecha)

# Plot netCDF
#plt.figure(figsize=(12,8))
#plt.pcolor(lon, lat, sst, cmap='jet')
#plt.colorbar()
#plt.clim(0, 30)
#plt.scatter(trayectories_df['longitude'], trayectories_df['latitude'], s=0.1, c='DarkBlue')
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.title(f"Sea Surface Temperature {fecha}")
#plt.show()


../data/sst/20140122090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1_clipped.nc
dict_keys(['time', 'lat', 'lon', 'analysed_sst', 'analysis_error', 'mask', 'sea_ice_fraction'])


In [11]:
# Obtener temperatura de netCDF a partir de coordenadas de puntos
variable_name = 'analysed_sst'
sst_onsite = get_values(lon, lat, sst,variable_name, puntos)
sst_onsite.head()


,id,geom,date,latitude,longitude,name,season,spheroid_dist_to_colony,sst,analysed_sst
1251,281,POINT(-118.291402 28.883883),2014-01-22,28.883883,-118.291405,LAAL01,incubacion,0.0,17.746,17.746
1252,282,POINT(-118.291442 28.883833),2014-01-22,28.883833,-118.291440,LAAL01,incubacion,0.0,17.746,17.746
1253,283,POINT(-118.291393 28.883812),2014-01-22,28.883812,-118.291390,LAAL01,incubacion,0.0,17.746,17.746
1254,284,POINT(-118.291473 28.883857),2014-01-22,28.883858,-118.291470,LAAL01,incubacion,0.0,17.746,17.746
1255,285,POINT(-118.29144 28.88382),2014-01-22,28.883820,-118.291440,LAAL01,incubacion,0.0,17.746,17.746


In [ ]:
# obten SST para todas las fechas

sst_all = pd.DataFrame()

for i in range(len(dates_df)):
    fecha = dates_df['date'].iloc[i]
    lon, lat, sst = select_netCDf(fecha)
    puntos = pd.DataFrame(get_points(fecha))
    sst_onsite = get_values(lon, lat, sst, puntos)
    sst_all = sst_all.append(sst_onsite)
    print(fecha)

In [ ]:
#sst_all.to_csv('../data/sst_all.csv', index=False)
sst_all[sst_all['sst'] == -9999] = np.nan

# convertir columna ssst a float con 3 decimales
sst_all['sst'] = sst_all['sst'].astype(float).round(3)
sst_all.describe()

# cuenta de valores nulos
sst_all.isnull().sum()

# Escritura de archivo csv
sst_all.to_csv('../data/sst_all.csv', index=False)
